In [2]:
from algorihtms import value_iteration, policy_iteration, Q_learning
from envs import game2048_env
import gym
import pickle
from matplotlib import pyplot as plt
import seaborn as sns

sns.set(color_codes=True)

In [3]:
%matplotlib inline

In [3]:
env = game2048_env.Game2048Env(size=2)
optimal_vs, optimal_ts, optimal_es = value_iteration.value_iteration(env, gamma=1, step=1000)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


AttributeError: 'Box' object has no attribute 'n'